## Importações Necessárias

In [1]:
!pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=6d0c35e0e3dd58c6071b653c6492610c49a54660421dff68606d8df74856338e
  Stored in directory: /root/.cache/pip/wheels/6e/62/11/dc73d78e40a218ad52e7451f30166e94491be013a7850b5d75
Successfully built fpdf


In [2]:
!pip install Mesa==0.5.1

  Preparing metadata (setup.py) ... done
  Created wheel for Mesa: filename=Mesa-0.5.1-py3-none-any.whl size=8467 sha256=64ca60ba60a3d929cc1e3dad907124fd4dda3180bbf4c54c7da2906b22e826b0
  Stored in directory: /root/.cache/pip/wheels/4a/20/83/9883468540627d03c3342a8e29728cf8ef81ff7a379e1556ab
Successfully built Mesa


In [3]:

import time
import json # Para lidar com a resposta JSON estruturada do LLM
import numpy as np
from fpdf import FPDF
from mesa import Agent, Model
from mesa.time import RandomActivation
import ast # Para parsear a string do LLM para dict

## Sistema de Desenvolvimento de Proposta Multiagente

In [5]:
# --- SIMULAÇÃO DA CHAMADA DE API DO GEMINI ---
# Em um ambiente real, você faria uma chamada POST para a API do Gemini
# com um systemInstruction e um responseSchema para garantir o formato JSON.
# Aqui, nesta PoC para iniciantes simulamos o resultado estruturado.

def simulate_gemini_structured_response(prompt_text):
    """
    Simula a chamada à API do Gemini para gerar um escopo e condições
    comerciais em formato JSON estruturado.

    Args:
        prompt_text: A descrição do serviço a ser analisada pelo LLM.

    Returns:
        Um dicionário Python (resultado da conversão do JSON) simulando
        a resposta estruturada do LLM.
    """
    # System Instruction: Define o papel do LLM para garantir a consistência
    # system_prompt = "Você é um Analista de Propostas Sênior. Sua tarefa é analisar a descrição do projeto e gerar um Escopo Técnico detalhado e Condições Comerciais em formato JSON estruturado."

    # Structured Response Schema (Simulado):
    # response_schema = {
    #   "scope_title": "Título do Escopo",
    #   "scope_summary": "Resumo executivo do escopo.",
    #   "scope_details": [{"item": "...", "description": "..."}, ...],
    #   "commercial_conditions": {"price_base": 50000.00, "payment_terms": "...", "warranty": "..."}
    # }

    print(f"LLMAgent: Analisando a solicitação: '{prompt_text}' e gerando estrutura...")
    time.sleep(3) # Simula o tempo de latência da API

    # Estrutura JSON simulada e consistente
    simulated_json_response = {
      "scope_title": "Proposta para Construção e Lançamento de Ponte Rodoviária de Médio Porte",
      "scope_summary": "Este escopo cobre todas as fases, desde o estudo de viabilidade até a entrega final da estrutura, focando em segurança, durabilidade e conformidade regulatória.",
      "scope_details": [
        {"item": "Fase 1: Estudo de Viabilidade e Licenciamento", "description": "Análise geotécnica, levantamento topográfico e obtenção de todas as licenças ambientais e de construção necessárias."},
        {"item": "Fase 2: Projeto Estrutural Detalhado", "description": "Desenvolvimento completo do projeto de engenharia, incluindo cálculos de carga, especificações de materiais (concreto e aço) e desenhos de execução."},
        {"item": "Fase 3: Execução da Obra e Lançamento", "description": "Construção das fundações (pilares e estacas), montagem da superestrutura e operação de lançamento da ponte. Inclui gestão de segurança rigorosa."},
        {"item": "Fase 4: Testes de Carga e Entrega Final", "description": "Realização de testes de estresse conforme normas ABNT e preparação da documentação 'As Built' para entrega ao cliente."}
      ],
      "commercial_conditions": {
        "price_base": 2850000.00,
        "payment_terms": "20% na assinatura; 40% após a conclusão da Fase 2; 40% na entrega e aceite final.",
        "warranty": "5 anos de garantia contra defeitos estruturais e de materiais."
      }
    }

    print("LLMAgent: Estrutura JSON gerada com sucesso.")
    return simulated_json_response

In [6]:
# --- Definição dos Agentes ---

class ScopeAgent(Agent):
    """Agente responsável por interagir com o LLM (Gemini) e gerar o escopo estruturado."""
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.scope_ready = False
        self.proposal_data = {} # Armazena o JSON estruturado do LLM

    def generate_scope(self, description):
        # Chama a função que simula a inteligência do LLM e obtém o JSON
        self.proposal_data = simulate_gemini_structured_response(description)
        self.scope_ready = bool(self.proposal_data)
        print("ScopeAgent: Dados da Proposta (LLM) armazenados.")

    def step(self, model):
        pass

class CostAgent(Agent):
    """Agente responsável por calcular os custos e refinar a proposta."""
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.costs_ready = False
        self.final_cost = 0.0

    def calculate_costs(self, proposal_data):
        if not proposal_data:
            return

        # O custo base veio do LLM (price_base), mas o Agente de Custo aplica a margem
        base_cost = proposal_data['commercial_conditions']['price_base']

        print("CostAgent: Calculando e ajustando custos...")
        time.sleep(1)

        # Simulação de cálculo de custo mais robusto: Custo base + 15% de imposto/margem
        margin_factor = 1.15
        self.final_cost = base_cost * margin_factor
        self.costs_ready = True
        print(f"CostAgent: Custo final calculado: R$ {self.final_cost:,.2f}")

    def step(self, model):
        pass

class PDFAgent(Agent):
    """Agente responsável por formatar e gerar o PDF final."""
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.pdf_generated = False

    def generate_pdf(self, proposal_data, final_cost, filename="proposta_gemini.pdf"):
        if not all([proposal_data, final_cost]):
            return
        print("PDFAgent: Gerando o arquivo PDF com conteúdo estruturado...")

        # Desempacota os dados estruturados
        scope_title = proposal_data['scope_title']
        scope_summary = proposal_data['scope_summary']
        scope_details = proposal_data['scope_details']

        # Desempacota as condições comerciais
        conditions = proposal_data['commercial_conditions']
        payment_terms = conditions['payment_terms']
        warranty = conditions['warranty']

        pdf = FPDF('P', 'mm', 'A4')
        pdf.set_auto_page_break(auto=True, margin=15)
        pdf.add_page()

        # Configuração de Título
        pdf.set_font("Arial", 'B', 16)
        pdf.cell(0, 10, txt="Proposta Técnica e Comercial - Gerada por Agentes LLM", ln=1, align="C")
        pdf.set_font("Arial", '', 10)
        pdf.cell(0, 5, txt="Estrutura de Proposta robusta e consistente", ln=1, align="C")
        pdf.ln(5)

        # Título do Escopo
        pdf.set_fill_color(220, 220, 220)
        pdf.set_font("Arial", 'B', 14)
        pdf.cell(0, 8, txt=scope_title, ln=1, fill=True, align='L')
        pdf.ln(2)

        # Resumo Executivo
        pdf.set_font("Arial", 'B', 12)
        pdf.cell(0, 6, txt="Resumo Executivo", ln=1)
        pdf.set_font("Arial", '', 11)
        pdf.multi_cell(0, 6, txt=scope_summary)
        pdf.ln(5)

        # Detalhes do Escopo
        pdf.set_font("Arial", 'B', 12)
        pdf.cell(0, 6, txt="Detalhes do Escopo:", ln=1)
        pdf.set_font("Arial", '', 11)

        for detail in scope_details:
            pdf.set_font("Arial", 'B', 11)
            # Encode to utf-8 to handle special characters like bullet points
            pdf.multi_cell(0, 6, txt=f"• {detail['item']}".encode('utf-8').decode('latin-1'), border=0, align='L')
            pdf.set_font("Arial", 'I', 10)
            # Encode to utf-8 to handle special characters like bullet points
            pdf.multi_cell(0, 5, txt=f"  {detail['description']}".encode('utf-8').decode('latin-1'), border=0, align='J')
            pdf.ln(1)

        pdf.ln(5)

        # Condições Comerciais
        pdf.set_font("Arial", 'B', 14)
        pdf.cell(0, 8, txt="Condições Comerciais", ln=1, fill=True, align='L')
        pdf.ln(2)

        pdf.set_font("Arial", 'B', 12)
        pdf.cell(40, 6, txt="Termos de Pagamento:", border=0)
        pdf.set_font("Arial", '', 12)
        pdf.cell(0, 6, txt=payment_terms, ln=1)

        pdf.set_font("Arial", 'B', 12)
        pdf.cell(40, 6, txt="Garantia:", border=0)
        pdf.set_font("Arial", '', 12)
        pdf.cell(0, 6, txt=warranty, ln=1)

        pdf.ln(5)

        # Custo Final
        pdf.set_font("Arial", 'B', 16)
        pdf.set_text_color(255, 0, 0)
        pdf.cell(0, 10, txt=f"CUSTO TOTAL FINAL: R$ {final_cost:,.2f}", ln=1, align="C", border=1)

        pdf.output(filename) # Output the PDF

        self.pdf_generated = True
        print(f"PDFAgent: Proposta gerada com sucesso: {filename}")

    def step(self, model):
        pass

In [7]:
class CoordinatorAgent(Agent):
    """Agente que coordena a solicitação e a comunicação entre os outros agentes."""
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.proposal_created = False
        # A descrição do projeto que seria analisada pelo LLM
        self.project_description = "Construção de uma ponte rodoviária de concreto armado de 150 metros sobre o rio principal."

    def step(self, model):
        # A lógica de coordenação acontece aqui
        if not self.proposal_created:

            # Recupera os agentes (simplesmente iterando para garantir que todos existam)
            scope_agent = next((a for a in self.model.schedule.agents if isinstance(a, ScopeAgent)), None)
            cost_agent = next((a for a in self.model.schedule.agents if isinstance(a, CostAgent)), None)
            pdf_agent = next((a for a in self.model.schedule.agents if isinstance(a, PDFAgent)), None)

            if scope_agent and cost_agent and pdf_agent:
                print(f"\nCoordinator: Iniciando geração da Proposta para: {self.project_description}")

                # 1. Solicita ao Agente de Escopo para interagir com o LLM (Gemini)
                scope_agent.generate_scope(self.project_description)

                # 2. Quando o escopo estruturado estiver pronto
                if scope_agent.scope_ready:

                    # 3. Solicita ao Agente de Custos que inicie o cálculo, passando o JSON estruturado
                    cost_agent.calculate_costs(scope_agent.proposal_data)

                    # 4. Quando o custo final for ajustado
                    if cost_agent.costs_ready:

                        # 5. Solicita ao Agente de PDF para compilar os dados e gerar a proposta final
                        pdf_agent.generate_pdf(
                            proposal_data=scope_agent.proposal_data,
                            final_cost=cost_agent.final_cost
                        )
                        self.proposal_created = True
                        print("\nCoordinator: Processo de Proposta Concluído.")
                        self.model.running = False # Encerra o modelo após a criação da proposta

class ProposalModel(Model):
    """Modelo multiagente para gerar propostas."""
    def __init__(self):
        self.schedule = RandomActivation(self)
        self.running = True

        # Cria os agentes
        self.coordinator = CoordinatorAgent(0, self)
        self.scope_agent = ScopeAgent(1, self)
        self.cost_agent = CostAgent(2, self)
        self.pdf_agent = PDFAgent(3, self)

        # Adiciona os agentes ao agendador
        self.schedule.add(self.coordinator)
        self.schedule.add(self.scope_agent)
        self.schedule.add(self.cost_agent)
        self.schedule.add(self.pdf_agent)

    def step(self):
        # Avança o modelo em um "passo" de tempo
        self.schedule.step()


In [8]:
# --- Execução do modelo ---

if __name__ == "__main__":
    print("Iniciando Simulação de Geração de Proposta Multiagente com LLM (Gemini)...")
    model = ProposalModel()
    while model.running:
        model.step()
    print("Simulação concluída!")


Iniciando Simulação de Geração de Proposta Multiagente com LLM (Gemini)...

Coordinator: Iniciando geração da Proposta para: Construção de uma ponte rodoviária de concreto armado de 150 metros sobre o rio principal.
LLMAgent: Analisando a solicitação: 'Construção de uma ponte rodoviária de concreto armado de 150 metros sobre o rio principal.' e gerando estrutura...
LLMAgent: Estrutura JSON gerada com sucesso.
ScopeAgent: Dados da Proposta (LLM) armazenados.
CostAgent: Calculando e ajustando custos...
CostAgent: Custo final calculado: R$ 3,277,500.00
PDFAgent: Gerando o arquivo PDF com conteúdo estruturado...
PDFAgent: Proposta gerada com sucesso: proposta_gemini.pdf

Coordinator: Processo de Proposta Concluído.
Simulação concluída!
